In [ ]:
!pip install mysql-connector-python
import mysql.connector
import sqlite3
import random
from datetime import datetime, timedelta

# ---------- MySQL DB Connection ----------
mysql_conn = mysql.connector.connect(
    host="127.0.0.1", user="root", password="", database="smartshow_db", port=3306
)
mysql_cursor = mysql_conn.cursor()
print("✅ Connected to MySQL")

# ---------- SQLite Connection (Movies) ----------
sqlite_conn = sqlite3.connect("movies")
sqlite_cursor = sqlite_conn.cursor()

# ---------- Validation ----------
def valid_email(email):
    return email.endswith("@gmail.com")

def valid_password(pw):
    return any(c.isupper() for c in pw) and any(c.islower() for c in pw) and "@" in pw

def generate_otp():
    return str(random.randint(100000, 999999))

# ---------- Register ----------
def register_user():
    name = input("Enter name: ")
    email = input("Enter email: ")
    if not valid_email(email):
        print("❌ Email must be @gmail.com"); return False

    mysql_cursor.execute("SELECT email FROM users WHERE email=%s", (email,))
    if mysql_cursor.fetchone():
        print("❌ Email already exists!"); return False

    pw = input("Enter password: ")
    if not valid_password(pw):
        print("❌ Password must have 1 uppercase, 1 lowercase & 1 '@'"); return False

    mysql_cursor.execute("SELECT password FROM users WHERE password=%s", (pw,))
    if mysql_cursor.fetchone():
        print("❌ Password already used! Choose another"); return False

    otp = generate_otp()
    expiry = datetime.now() + timedelta(minutes=2)

    mysql_cursor.execute(
        "INSERT INTO users (name,email,password,otp,otp_expiry) VALUES (%s,%s,%s,%s,%s)",
        (name,email,pw,otp,expiry)
    )
    mysql_conn.commit()
    print(f"🔐 Your OTP: {otp} (valid 2 min)")

    user_otp = input("Enter OTP: ")
    if user_otp != otp:
        print("❌ Invalid OTP"); return False

    mysql_cursor.execute("UPDATE users SET otp=NULL,otp_expiry=NULL WHERE email=%s",(email,))
    mysql_conn.commit()
    print("✅ Registered successfully")
    return True

# ---------- Login ----------
def login_user():
    email = input("Enter email: ")
    pw = input("Enter password: ")
    mysql_cursor.execute("SELECT * FROM users WHERE email=%s AND password=%s",(email,pw))
    if mysql_cursor.fetchone():
        print("✅ Login successful"); return True
    print("❌ Invalid email or password"); return False

# ---------- Theater Class ----------
class Theater:
    def __init__(self, theater_id, name, area, city, theater_type, total_screens):
        self.theater_id = theater_id
        self.name = name
        self.area = area
        self.city = city
        self.theater_type = theater_type
        self.total_screens = total_screens

    def show_details(self):
        print("\n--- Theatre Details ---")
        print(f"ID      : {self.theater_id}")
        print(f"Name    : {self.name}")
        print(f"Area    : {self.area}")
        print(f"City    : {self.city}")
        print(f"Type    : {self.theater_type}")
        print(f"Screens : {self.total_screens}")

# ---------- Show First 20 Theatres ----------
def show_first_20_theatres():
    print("\n========== ALL THEATRES (FIRST 20) ==========")
    mysql_cursor.execute("SELECT * FROM theatres LIMIT 20")
    rows = mysql_cursor.fetchall()
    for row in rows:
        t = Theater(*row)
        t.show_details()

# ---------- Movie Class ----------
class Movie:
    def __init__(self):
        self.movie_theatre_map = {}  # movie_id → fixed 5 theatres

    def movies(self):
        while True:
            print("\nChoose Your Mood")
            print("1. Romantic  2. Relaxed  3. Excited  4. Family  0. Exit")
            mood_map = {"1":"Romantic","2":"Relaxed","3":"Excited","4":"Family"}
            choice = input("Enter choice: ")

            if choice=="0":
                print("👋 Thank you for using SmartShow!"); break

            mood = mood_map.get(choice)
            if not mood:
                print("❌ Invalid choice, try again."); continue

            # Fetch 10 movies for selected mood
            sqlite_cursor.execute("SELECT id,movie_name FROM movies WHERE mood=? LIMIT 10",(mood,))
            movies = sqlite_cursor.fetchall()
            if not movies:
                print("❌ No movies found"); continue

            print("\n🎬 Movies List:")
            allowed_movie_ids = []
            for m in movies:
                print(f"{m[0]}. {m[1]}")
                allowed_movie_ids.append(str(m[0]))

            movie_id = input("\nSelect Movie ID: ")
            if movie_id not in allowed_movie_ids:
                print("❌ Invalid Movie ID for this mood!"); continue

            # Assign 5 fixed theaters for movie
            if movie_id not in self.movie_theatre_map:
                mysql_cursor.execute("SELECT theater_id FROM theatres")
                all_ids = [row[0] for row in mysql_cursor.fetchall()]
                self.movie_theatre_map[movie_id] = random.sample(all_ids,5)

            fixed_ids = self.movie_theatre_map[movie_id]

            mysql_cursor.execute(
                f"SELECT * FROM theatres WHERE theater_id IN ({','.join(map(str,fixed_ids))})"
            )
            rows = mysql_cursor.fetchall()
            print("\n🎭 Theatres Showing This Movie:")
            theaters = [Theater(*r) for r in rows]
            for t in theaters:
                print(f"{t.theater_id}. {t.name} ({t.city})")

            # Theatre selection with validation
            while True:
                tid = input("Enter Theatre ID to view details: ")
                if tid not in [str(i) for i in fixed_ids]:
                    print("❌ Invalid Theatre ID for this movie! Re-enter."); continue
                break

            mysql_cursor.execute("SELECT * FROM theatres WHERE theater_id=%s",(tid,))
            res = mysql_cursor.fetchone()
            if res:
                t = Theater(*res)
                t.show_details()
            else:
                print("❌ Theatre details not found!")

# ---------- Main Program ----------
print("====================================================================")
print("        Welcome To Our SmartShow! Enjoy Your Day 🎬              ")
print("====================================================================")

# Register/Login once
while True:
    print("\n1. Register  2. Login")
    choice = input("Choose option: ")
    success = False
    if choice=="1":
        success = register_user()
    elif choice=="2":
        success = login_user()
    else:
        print("❌ Invalid choice"); continue

    if success:
        break

# Show first 20 theatres and start movie selection
show_first_20_theatres()
obj = Movie()
obj.movies()

# Close DB connections
sqlite_cursor.close()
sqlite_conn.close()
mysql_cursor.close()
mysql_conn.close()
